In [8]:
# https://github.com/taki0112/ResNet-Tensorflow
! python3 resnet_main.py 2> /dev/null

W1105 19:33:20.449201 139990289348352 deprecation_wrapper.py:119] From resnet_main.py:25: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1105 19:33:20.449403 139990289348352 deprecation_wrapper.py:119] From resnet_main.py:27: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-11-05 19:33:20.449715: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-11-05 19:33:20.454910: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-11-05 19:33:20.509463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-11-05 19:33:20.510149: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x475be40 executing computations